This notebook has example code for fetching WB grouping hierarchy from FMR PROD: https://fmr.worldbank.org/FMR

In [3]:
import requests
import json
import pandas as pd

## WB groups hierarchy
Hierarchy contains the relations between each groups and areas.

Hierarchy ID: H_REF_AREA_GROUPS  
Version: 
- V38.0 (for FY26 - postMENA chanegs)
- V37.0 (for FY25 - preMENA changes)
- V36.0 (for FY24)  
...


In [5]:
grouping_version = "38.0"
fmr_prod_base_url = "https://fmr.worldbank.org/FMR/sdmx/v2/structure/"

ref_area_groups_url = f"{fmr_prod_base_url}hierarchy/WB/H_REF_AREA_GROUPS/{grouping_version}?format=fusion-json"

In [10]:
# Extracting hierarchical codes from hierarchy
response = requests.get(ref_area_groups_url).json()

code_data = []
for item in response["Hierarchy"][0]["codes"]:
    for sub_item in item["codes"]:
        if "codes" in sub_item:
            for sub2_item in sub_item["codes"]:
                code_data.append({"Code_URN": sub2_item["urn"].split(")")[-1]})

hierarchy = pd.DataFrame(code_data)
hierarchy_split = (
    hierarchy["Code_URN"].str.split(".", expand=True).drop(columns=[0])
)
hierarchy_split = hierarchy_split.rename(
    columns={1: "Group Type", 2: "Group", 3: "Economy in group"}
)
hierarchy_split

,Group Type,Group,Economy in group
0,REGION,AFE,AGO
1,REGION,AFE,BWA
2,REGION,AFE,BDI
3,REGION,AFE,COM
4,REGION,AFE,COD
...,...,...,...
4401,CONTINENT,009,WSM
4402,CONTINENT,009,TKL
4403,CONTINENT,009,TON
4404,CONTINENT,009,TUV


## WB groups codelist
Codelist contains the ID and name for each area.

Codelist ID: WB:CL_AREA  
Version: V2.0 

In [12]:
name_version = "2.0"
fmr_prod_base_url = "https://fmr.worldbank.org/FMR/sdmx/v2/structure/"

ref_area_name_url = f"{fmr_prod_base_url}codelist/WB/CL_AREA/{name_version}?format=fusion-json"

In [13]:
# Extracting hierarchical names from codelist
response = requests.get(ref_area_name_url).json()

name_data = []
for item in response["Codelist"]:
    for sub_item in item["items"]:
        name_data.append(
            {"Code": sub_item["id"], "Value": sub_item["names"][0]["value"]}
        )

name = pd.DataFrame(name_data)
name 

,Code,Value
0,ESCOM,Electricity Supply Corporation of Malawi (ESCOM)
1,ESKOM,Eskom (ESKOM)
2,WLD,World
3,DFS,IDA countries classified as fragile situations
4,BSA,South Asia (IBRD only)
...,...,...
573,THI,High income (IDA & IBRD)
574,TLM,Lower middle income (IDA & IBRD)
575,TUM,Upper middle income (IDA & IBRD)
576,KEN_POW,Kenya Power and Lighting Company (Kenya Pow)


In [20]:
# Merge the grouping hierarchy and name
grouping = pd.merge(hierarchy_split, name, left_on="Group", right_on="Code", how="left")
grouping = pd.merge(grouping, name, left_on="Economy in group", right_on="Code", how="left")
grouping = grouping.drop(columns=["Code_x", "Code_y"])
grouping.rename(
    columns={
        "Value_x": "Group name",
        "Value_y": "Economy in group name",
    },
    inplace=True,
)
grouping = grouping[['Group Type', 'Group', 'Group name', 'Economy in group', 'Economy in group name']]
grouping

,Group Type,Group,Group name,Economy in group,Economy in group name
0,REGION,AFE,Eastern & Southern Africa,AGO,Angola
1,REGION,AFE,Eastern & Southern Africa,BWA,Botswana
2,REGION,AFE,Eastern & Southern Africa,BDI,Burundi
3,REGION,AFE,Eastern & Southern Africa,COM,Comoros
4,REGION,AFE,Eastern & Southern Africa,COD,"Congo, Dem. Rep."
...,...,...,...,...,...
4401,CONTINENT,009,Oceania,WSM,Samoa
4402,CONTINENT,009,Oceania,TKL,Tokelau (NZ)
4403,CONTINENT,009,Oceania,TON,Tonga
4404,CONTINENT,009,Oceania,TUV,Tuvalu


In [25]:
print(f"Group Type: {grouping['Group Type'].unique()}")
print(f"Group name missing: {grouping['Group name'].isna().sum()}")
print(f"Economy in group name missing: {grouping['Economy in group name'].isna().sum()}")

Group Type: ['REGION' 'REGION_UN' 'LENDING' 'INCOME' 'OTHER' 'CONTINENT']
Group name missing: 0
Economy in group name missing: 0


## Strict WB region and income group
This will fetch region and income group without variation (i.e. IBRD only, excluding high income, IDA total)  

Region hierarchy ID: H_WB_REGIONS     
    ['AFE', 'AFW', 'EAS', 'ECS', 'LCN', 'MEA', 'NAC', 'SAS', 'SSF']  
    
Income group hierarchy ID: H_WB_INCOME  
    ['HIC', 'LIC', 'LMC', 'UMC', 'INX']    

In [ ]:
region_hierarchy_version = '2.0'
income_hierarchy_version = '2.0'
fmr_prod_base_url = "https://fmr.worldbank.org/FMR/sdmx/v2/structure/"

region_hierarchy_url = f"{fmr_prod_base_url}hierarchy/WB/H_WB_REGIONS/{region_hierarchy_version}?format=fusion-json"
income_hierarchy_url = f"{fmr_prod_base_url}hierarchy/WB/H_WB_INCOME/{income_hierarchy_version}?format=fusion-json"

In [ ]:
# For region
response = requests.get(region_hierarchy_url).json()

region_code = []
for item in response["Hierarchy"][0]["codes"]:
    for sub_item in item["codes"]:
        region_code.append(sub_item["id"])
region_code

In [ ]:
# For income group
response = requests.get(income_hierarchy_url).json()
income_code = []
for item in response["Hierarchy"][0]["codes"]:
    income_code.append(item["id"])
income_code

In [ ]:
wb_cl_long = get_wb_classifications('37.0')

# For region and income group, only keep value in region_code and income_code
wb_cl_long = wb_cl_long[~((wb_cl_long['group'] == 'REGION') & (~wb_cl_long['value'].isin(region_code)))]
wb_cl_long = wb_cl_long[~((wb_cl_long['group'] == 'INCOME') & (~wb_cl_long['value'].isin(income_code)))]

# Pivot to wide format
wide = wb_cl_long.pivot_table(index="ISO3", columns="group", values="value", aggfunc=lambda x: ','.join(x)).reset_index()
type_to_keep = ['CONTINENT', 'REGION', 'INCOME']
wb_cl_wide = wide[["ISO3"] + type_to_keep]
wb_cl_wide

In [ ]:
# Final function to get strict WB classifications in wide format
def get_wb_classifications_strict(
    grouping_version="38.0",
    region_version="2.0",
    income_version="2.0",
    fmr_prod_base_url="https://fmr.worldbank.org/FMR/sdmx/v2/structure/",
    type_to_keep=("CONTINENT", "REGION", "INCOME"),
):
    """
    Fetch World Bank classifications (continent, region, income) mapped to ISO3 codes.

    Parameters
    ----------
    grouping_version : str, optional
        Version of the World Bank reference area groups hierarchy. Default is "38.0"
        (corresponding to FY26).
    region_version : str, optional
        Version of the World Bank regions hierarchy. Default is "2.0".
    income_version : str, optional
        Version of the World Bank income groups hierarchy. Default is "2.0".
    fmr_prod_base_url : str, optional
        Base URL for accessing the World Bank FMR SDMX structures. Default is
        "https://fmr.worldbank.org/FMR/sdmx/v2/structure/".
    type_to_keep : tuple of str, optional
        Classification types to keep in the final table. Default is
        ("CONTINENT", "REGION", "INCOME").

    Returns
    -------
    pandas.DataFrame
        A wide-format dataframe containing ISO3 country codes as rows and selected
        classification types (e.g., CONTINENT, REGION, INCOME) as columns.
    """

    ref_area_groups_url = (
        f"{fmr_prod_base_url}hierarchy/WB/H_REF_AREA_GROUPS/{grouping_version}?format=fusion-json"
    )
    region_hierarchy_url = (
        f"{fmr_prod_base_url}hierarchy/WB/H_WB_REGIONS/{region_version}?format=fusion-json"
    )
    income_hierarchy_url = (
        f"{fmr_prod_base_url}hierarchy/WB/H_WB_INCOME/{income_version}?format=fusion-json"
    )

    # Parse H_REF_AREA_GROUPS hierarchy
    response = requests.get(ref_area_groups_url).json()
    code_data = [
        {"Code_URN": sub2["urn"].split(")")[-1]}
        for item in response["Hierarchy"][0]["codes"]
        for sub in item.get("codes", [])
        for sub2 in sub.get("codes", [])
    ]

    # Split URNs into structured columns
    res_split = (
        pd.DataFrame(code_data)["Code_URN"]
        .str.split(".", expand=True)
        .rename(columns={1: "group", 2: "value", 3: "ISO3"})
        .drop(columns=[0])
    )

    # Get valid region codes 
    region_response = requests.get(region_hierarchy_url).json()
    region_codes = [
        sub["id"]
        for item in region_response["Hierarchy"][0]["codes"]
        for sub in item.get("codes", [])
    ]

    # Get valid income codes 
    income_response = requests.get(income_hierarchy_url).json()
    income_codes = [item["id"] for item in income_response["Hierarchy"][0]["codes"]]

    # Filter invalid REGION / INCOME values 
    filters = {
        "REGION": set(region_codes),
        "INCOME": set(income_codes),
    }
    for grp, valid_values in filters.items():
        res_split = res_split[
            ~((res_split["group"] == grp) & (~res_split["value"].isin(valid_values)))
        ]

    # Pivot to wide format
    wide = (
        res_split.pivot_table(
            index="ISO3",
            columns="group",
            values="value",
            aggfunc=lambda x: ",".join(sorted(set(x))),
        )
        .reset_index()
    )

    # Keep only requested groups
    return wide[["ISO3"] + [col for col in type_to_keep if col in wide.columns]]